In [1]:
import cv2
from matplotlib import pyplot as plt
import os
import shutil
import numpy as np
from PIL import Image
import torch
from sklearn.model_selection import train_test_split

# tensorflow library
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
current_dir = os.getcwd()
print("Current Working Directory:", current_dir)
# new_directory = '/content/drive/My Drive/IT1244/experimental'
new_directory = '../experimental'
os.chdir(new_directory)

# Verify the change
print("New Current Working Directory:", os.getcwd())

Current Working Directory: g:\My Drive\Personal folder\My NUS folder\Year 2\Year 2 sem 1\IT1244\IT1244_Project\experimental
New Current Working Directory: g:\My Drive\Personal folder\My NUS folder\Year 2\Year 2 sem 1\IT1244\IT1244_Project\experimental


In [13]:
classes=set()

if not os.listdir("data_classes"):
    for image in os.listdir("../images"):
        classes.add(image[:2])

    #the following code extracts the numerical part of the class name
    #this is because some of the numeric values are 1 digit and will leave an underscore behind in the string
    #this would filter out the numeric portion
    newlist=[int(''.join(filter(str.isdigit, classname))) for classname in classes]


    parent_dir="../experimental/data_classes"
    imagePath="../images"

    for num in (newlist):
        dir_path = os.path.join(parent_dir, str(num))
        os.makedirs(dir_path, exist_ok=True)
        print(f"Directory {dir_path} created.")


    for imagefilename in os.listdir(imagePath):
        newstring = "".join(filter(str.isdigit, imagefilename[:2]))
        targetdirectory=os.path.join(parent_dir, newstring)
        newimagepath=os.path.join(imagePath, imagefilename)
        shutil.copy(newimagepath, targetdirectory)
        print("image moved")

Directory ../experimental/data_classes\9 created.
Directory ../experimental/data_classes\19 created.
Directory ../experimental/data_classes\8 created.
Directory ../experimental/data_classes\10 created.
Directory ../experimental/data_classes\6 created.
Directory ../experimental/data_classes\21 created.
Directory ../experimental/data_classes\14 created.
Directory ../experimental/data_classes\11 created.
Directory ../experimental/data_classes\3 created.
Directory ../experimental/data_classes\20 created.
Directory ../experimental/data_classes\18 created.
Directory ../experimental/data_classes\12 created.
Directory ../experimental/data_classes\1 created.
Directory ../experimental/data_classes\5 created.
Directory ../experimental/data_classes\0 created.
Directory ../experimental/data_classes\22 created.
Directory ../experimental/data_classes\13 created.
Directory ../experimental/data_classes\2 created.
Directory ../experimental/data_classes\16 created.
Directory ../experimental/data_classes\

In [32]:
add_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2
)

base_dir = '../experimental/data_classes'
#choose 514 because the max number of images among the classes is 514
target_count = 400
for class_num in range(23):  
    class_dir = os.path.join(base_dir, str(class_num))
    images = os.listdir(class_dir)
    current_count = len(images)
    
    if current_count < target_count:
        print(f"Augmenting class {class_num} with {target_count - current_count} new images.")
        
        #this is because we want to top out the number of images in each class folders from 0 to  22
        images_needed = target_count - current_count
        
        # Augment images
        for img_name in images:
            img_path = os.path.join(class_dir, img_name)
            img = load_img(img_path) 
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)  
            
            # Generate new images
            generated_images = 0  # Track images generated for this specific image
            for batch in add_datagen.flow(x, batch_size=1, save_to_dir=class_dir, save_format='png'):
                generated_images += 1
                current_count += 1  # Update the current image count for the class
                
                # Stop generating if the target count is reached
                if current_count >= target_count:
                    break
            
            # If the target count is reached for this class, stop further augmentation
            if current_count >= target_count:
                break

    elif current_count > target_count:
        # Remove excess images to reach the target count
        images_to_remove = current_count - target_count
        print(f"Reducing class {class_num} by removing {images_to_remove} images.")
        
        # Randomly select images to remove
        images_to_delete = random.sample(images, images_to_remove)
        for img_name in images_to_delete:
            img_path = os.path.join(class_dir, img_name)
            os.remove(img_path)


Reducing class 0 by removing 114 images.
Reducing class 1 by removing 114 images.
Reducing class 2 by removing 114 images.
Reducing class 3 by removing 114 images.
Reducing class 4 by removing 114 images.
Reducing class 5 by removing 113 images.
Reducing class 6 by removing 114 images.
Reducing class 7 by removing 114 images.
Reducing class 8 by removing 114 images.
Reducing class 9 by removing 112 images.
Reducing class 10 by removing 114 images.
Reducing class 11 by removing 114 images.
Reducing class 12 by removing 114 images.
Reducing class 13 by removing 114 images.
Reducing class 14 by removing 114 images.
Reducing class 15 by removing 113 images.
Reducing class 16 by removing 114 images.
Reducing class 17 by removing 114 images.
Reducing class 18 by removing 114 images.
Reducing class 19 by removing 113 images.
Reducing class 20 by removing 114 images.
Reducing class 21 by removing 114 images.
Reducing class 22 by removing 114 images.


In [33]:
# Verify the number of images in each class directory
for class_num in range(23):
    class_dir = os.path.join(base_dir, str(class_num))
    images = os.listdir(class_dir)
    print(f"Class {class_num} now has {len(images)} images.")

Class 0 now has 400 images.
Class 1 now has 400 images.
Class 2 now has 400 images.
Class 3 now has 400 images.
Class 4 now has 400 images.
Class 5 now has 400 images.
Class 6 now has 400 images.
Class 7 now has 400 images.
Class 8 now has 400 images.
Class 9 now has 400 images.
Class 10 now has 400 images.
Class 11 now has 400 images.
Class 12 now has 400 images.
Class 13 now has 400 images.
Class 14 now has 400 images.
Class 15 now has 400 images.
Class 16 now has 400 images.
Class 17 now has 400 images.
Class 18 now has 400 images.
Class 19 now has 400 images.
Class 20 now has 400 images.
Class 21 now has 400 images.
Class 22 now has 400 images.


In [34]:
# Original directory containing folders 0 to 22
original_dir = '../experimental/data_classes'
train_dir = '../experimental/new/train'
val_dir = '../experimental/new/validation'
test_dir = '../experimental/new/test'

# Create directories for train, validation, and test splits
for i in range(23):  # 23 classes from 0 to 22
    os.makedirs(os.path.join(train_dir, str(i)), exist_ok=True)
    os.makedirs(os.path.join(val_dir, str(i)), exist_ok=True)
    os.makedirs(os.path.join(test_dir, str(i)), exist_ok=True)

# Split images in each class
for i in range(23):
    class_path = os.path.join(original_dir, str(i))
    images = os.listdir(class_path)
    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=1/3, random_state=42)

    # Copy images to the respective directories
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, str(i)))
    for img in val_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, str(i)))
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_dir, str(i)))


KeyboardInterrupt: 

In [ ]:
datagen = ImageDataGenerator(rescale=1./255) 

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),  # Set to your model’s input size
    batch_size=64,
    class_mode='sparse',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128,128),
    batch_size=64,
    class_mode='sparse',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=64,
    class_mode='sparse',
    shuffle=True
)

Found 9288 images belonging to 23 classes.
Found 3115 images belonging to 23 classes.
Found 1634 images belonging to 23 classes.


In [23]:
def testing_model(model, test, class_of_interest=6):
    y_true = []
    y_pred = []

    # Loop through test dataset and gather predictions and true labels
    for images, labels in test:
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))  # Predicted labels
        y_true.extend(labels.numpy())  # True labels

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate false negatives for the specified class
    false_negatives = np.sum((y_true == class_of_interest) & (y_pred != class_of_interest))
    total_positives = np.sum(y_true == class_of_interest)

    # Calculate false negative rate
    false_negative_rate = false_negatives / total_positives if total_positives > 0 else 0

    test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"False Negative Rate for class {class_of_interest}: {false_negative_rate:.4f}")

In [24]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [25]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.GaussianNoise(0.1)
])

In [30]:
model = Sequential([
    Input(shape=(128, 128, 3)),
    data_augmentation,
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(23, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       803,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 23)             │         5,911 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 818,695 (3.12 MB)

 Trainable params: 818,695 (3.12 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
hist = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    verbose=1
)

Epoch 1/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 78s 512ms/step - accuracy: 0.0491 - loss: 3.3823 - val_accuracy: 0.0648 - val_loss: 3.1332
Epoch 2/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 80s 538ms/step - accuracy: 0.0526 - loss: 3.1348 - val_accuracy: 0.0562 - val_loss: 3.1320
Epoch 3/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 84s 564ms/step - accuracy: 0.0528 - loss: 3.1344 - val_accuracy: 0.0626 - val_loss: 3.1313
Epoch 4/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 80s 534ms/step - accuracy: 0.0518 - loss: 3.1344 - val_accuracy: 0.0591 - val_loss: 3.1307
Epoch 5/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 80s 537ms/step - accuracy: 0.0495 - loss: 3.1349 - val_accuracy: 0.0591 - val_loss: 3.1307
Epoch 6/10
127/146 ━━━━━━━━━━━━━━━━━━━━ 8s 451ms/step - accuracy: 0.0500 - loss: 3.1341

KeyboardInterrupt: 